Load the CSV files into DataFrames using pandas.

In [ ]:
import pandas as pd
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')
sample_submission_df = pd.read_csv('sample_submission.csv')

Initialize the tokenizer and fit it on the training text data.

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])

Convert the test text data to sequences using the fitted tokenizer.

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_df['text'])

Pad the test sequences to ensure uniform input size.

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_length = 100  # Define maximum length

test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

Create a random embedding matrix for the vocabulary.

In [ ]:
import numpy as np

embedding_matrix = np.random.rand(len(tokenizer.word_index) + 1, 100)  # Random embedding matrix

Convert training labels to categorical format for model training.

In [ ]:
from keras.utils import to_categorical

train_labels_categorical = to_categorical(train_df['label'])

Define the CNN model structure with an embedding layer and convolutional layers.

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=len(train_df['label'].unique()), activation='softmax'))

Compile the model with appropriate loss function and optimizer.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model on the training data.

In [ ]:
model.fit(X_train, train_labels_categorical, epochs=5, batch_size=32)

Make predictions on the padded test data.

In [ ]:
predicted_labels = model.predict(test_padded)

Convert predicted probabilities to class labels using argmax.

In [ ]:
test_labels = np.argmax(predicted_labels, axis=1)